# **Pre-trained model for STT (Whisper) - common voice dataset**
Please refer to the respective sections in the book for further details.


## **Step 1. Installing Libraries and Data loading**

In [ ]:
!pip install jiwer
!pip install datasets
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.5 MB/s eta 0:00:00


In [ ]:
from jiwer import wer
import torch
from transformers import pipeline
from datasets import load_dataset, DatasetDict
import re

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
common_voice = DatasetDict()

common_voice_test_subset = load_dataset("mozilla-foundation/common_voice_13_0", "mrj", split="test")
common_voice["test"] = common_voice_test_subset

print(common_voice)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1454: FutureWarning: The repository for mozilla-foundation/common_voice_13_0 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/mozilla-foundation/common_voice_13_0
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the ne

Generating train split: 0 examples [00:00, ? examples/s]


Reading metadata...: 7272it [00:00, 136856.70it/s]


Generating validation split: 0 examples [00:00, ? examples/s]


Reading metadata...: 3297it [00:00, 118246.90it/s]


...

Generating other split: 0 examples [00:00, ? examples/s]


Reading metadata...: 10180it [00:00, 143976.20it/s]


Generating invalidated split: 0 examples [00:00, ? examples/s]


Reading metadata...: 272it [00:00, 60076.39it/s]

DatasetDict({
    test: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment', 'variant'],
        num_rows: 4428
    })
})


In [ ]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
pipe = pipeline("automatic-speech-recognition", model="openai/whisper-small", device=device)

config.json:   0%|          | 0.00/1.97k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.87k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/805 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/836k [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.08k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

## **Step 2. Model Inference**

In [ ]:
common_voice = common_voice.select_columns(["audio", "sentence"])

In [ ]:
def normalize_text(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    return text

## **Step 3. Model Evaluation**

In [ ]:
total_wer = 0
for sample in common_voice["test"]:
    reference = sample["sentence"]
    reference = normalize_text(reference)
    hypothesis = pipe(sample["audio"].copy(), generate_kwargs={"task": "transcribe"})
    hypothesis = normalize_text(hypothesis['text'])
    total_wer += wer(reference, hypothesis)
    print(f"Reference: {reference}")
    print(f"Hypothesis: {hypothesis}")
    print(f"WER: {wer(reference, hypothesis)}")

Reference: колжы миде сеткӓшкӹ
Hypothesis:  koža midė sėt keška
WER: 1.3333333333333333
Reference: йынгы йыла стихотворени отважный морякреволюционер макаров лӹмеш сирӹмӹ
Hypothesis:  янга ела стихотворение отважный моряк революционер макаров лемеш сирами
WER: 0.875
Reference: кырыквлӓ ӓнгӹрвлӓм йоктарат
Hypothesis:  كاراكولا أنغر بلم يكتارات
WER: 1.3333333333333333
Reference: а микитӓм ӓштет тӹнь 
Hypothesis:  아 비키템 에스티트잉
WER: 1.0
Reference: райжы вет уке ылеш
Hypothesis:  raži vietų keliš
WER: 1.0
Reference: кыдыжы ак кердеп вӓк качкын 
Hypothesis:  kad žaukirti vėg kačkai
WER: 1.0
Reference: яратым шаным тӓнг
Hypothesis:  я рад им шанам тенг
WER: 1.6666666666666667
Reference: тӹнь маняр дивизим 
Hypothesis:  тань маняр диризим
WER: 0.6666666666666666
Reference: анжал колтет  уже кӹцкӹ шӹтӓ
Hypothesis:  angel coltiet uz ekatiskaša te
WER: 1.0


KeyboardInterrupt: 

In [ ]:
total_length = len(common_voice['test'])

4428


In [ ]:
average_wer = total_wer / total_length
print(f"Average WER: {average_wer}")

Average WER: 1.291118148638472
